In [ ]:
%matplotlib inline
import quest
import numpy as np 
from matplotlib import colors, cm, pyplot as plt
from gazar import grid

In [ ]:
VICKSBURG = 'Vicksburg'
PROVO = 'Provo'

location = PROVO
# location = VICKSBURG

SERVICE_FEATURES = {VICKSBURG: 'svc://usgs-ned:13-arc-second/581d213be4b08da350d52d69', #581d213be4b08da350d52d67',
                    PROVO: 'svc://usgs-ned:13-arc-second/581d2162e4b08da350d5325e'
                    }
BBOXES = {VICKSBURG: [-90.9, -90.8, 32.2, 32.3],
          PROVO: [-111.6, -111.4, 40.0, 40.15]
         }

OUTLETS = {VICKSBURG: (-90.889, 32.2133),
           PROVO: (-111.44851851851851, 40.0787962962963),
          }

service_feature = SERVICE_FEATURES[location]
bbox = BBOXES[location]
outlets = [OUTLETS[location]]

In [ ]:
# create new collection
# Avoid re-downloading elevation data if already present 
collection = 'watershed_delineation'
elev_name = 'merged elevation raster ' + location
elevation = quest.api.get_datasets(filters={'display_name': elev_name})

if not elevation:
    try:
        quest.api.new_collection(collection)
    except ValueError:
        quest.api.delete(collection)
        quest.api.new_collection(collection)

    # download elevation data
    feature = quest.api.add_features(collection, service_feature)
    dataset = quest.api.stage_for_download(feature)
    quest.api.download_datasets(dataset)
    result = quest.api.apply_filter('raster-merge', datasets=dataset, options={'bbox': bbox}, display_name=elev_name)
    elevation = result['datasets'][0]

In [ ]:
# run pit filling algorithm
algorithm = 'go-fill'  # one of ['flats', 'go-fill', 'go-breach']
result = quest.api.apply_filter('raster-fill', datasets=elevation, options={'algorithm': algorithm})
elevation = result['datasets'][0]

In [ ]:
# run flow accumulation
algorithm = 'go-d8'  # one of ['d8', 'go-d8', 'go-fd8']
result = quest.api.apply_filter('raster-flow-accumulation', datasets=elevation, options={'algorithm': algorithm})
flow_accumulation_dataset_id = result['datasets'][0]

In [ ]:
# read in flow accumulation
flow_accumulation_dataset_file = quest.api.get_metadata(flow_accumulation_dataset_id)[flow_accumulation_dataset_id]['file_path']
flow_accumulation_dataset = grid.GDALGrid(flow_accumulation_dataset_file)
flow_accumulation_data = flow_accumulation_dataset.np_array()

max_accumulation = flow_accumulation_data.max()
stream_threshold = max_accumulation * 0.01
rivers = np.ma.masked_where(flow_accumulation_data < stream_threshold, flow_accumulation_data)

In [ ]:
# read in elevation data
dem_file = quest.api.get_metadata(elevation)[elevation]['file_path']
dem = grid.GDALGrid(dem_file)
lat, lon = dem.latlon

In [ ]:
result = quest.api.apply_filter('raster-watershed-delineation', datasets=elevation, 
                       options={'outlet_points': outlets, 'snap_outlets': 'jenson', 'stream_threshold_pct': 0.01})
watershed = result['features']['watershed']
outlet = result['features']['outlet']
watershed_geometry = quest.api.get_metadata(watershed)[watershed]['geometry']
outlet_lon, outlet_lat = quest.api.get_metadata(outlet)[outlet]['geometry'].coords.xy
print(watershed_geometry.area)
watershed_geometry

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

display_bbox = bbox
graticule_spacing = 0.01

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(display_bbox)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabels_top = False

plt.contourf(lon, lat, dem.np_array(), 100, zorder=2, cmap=cm.terrain, transform=ccrs.PlateCarree())
ax.imshow(rivers, cmap=cm.ocean, origin='lower', zorder=3)
watershed_shp = cfeature.ShapelyFeature([watershed_geometry], ccrs.PlateCarree())
ax.add_feature(watershed_shp, zorder=4, alpha=0.6)
ax.scatter(x=outlet_lon, y=outlet_lat, color='red', zorder=5, transform=ccrs.PlateCarree())
plt.title("Watershed")
plt.show()